# newer crf trained with kfolded data

In [1]:
#Data Analysis
import pandas as pd
import numpy as np

#Data Visualisation
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)
sns.set(font_scale=1)
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

#Modeling
import fasttext
from sklearn.model_selection import cross_val_predict, cross_val_score, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn_crfsuite import CRF, scorers, metrics
from sklearn_crfsuite.metrics import flat_classification_report
from sklearn.metrics import classification_report, make_scorer
import scipy.stats
import eli5

In [4]:
path = "C:\\Users\\farih\\Downloads\\"
indoNLUmodel = "C:\\Users\\farih\\Downloads\\fasttext.4B.id.300.epoch5.uncased.bin"
model = fasttext.load_model(indoNLUmodel)

In [54]:
train_df = pd.read_csv("D:\Drive\KULEEAH\Semester 8\TA Farihin\Dataset\\StratifiedKFold\\trainval.csv")
test_df = pd.read_csv("D:\Drive\KULEEAH\Semester 8\TA Farihin\Dataset\\StratifiedKFold\\test.csv")

In [55]:
train_df.head(1)

,tweet_id,token,ner,token_idx,ner_idx,entities,count_entities,onehot_entities
0,1380681275594305537,"['@ridwankamil', '@KickAndyShow', '@Metro_TV',...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ...","[17441, 14696, 23089, 29865, 5288, 16801, 4219...","[12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 12, 1...","[['gubernur', 'PER'], ['jabar', 'LOC']]","[Counter({'PER': 1, 'LOC': 1})]",110000


In [56]:
def getsentence(df):
    def _zip(a,b):
            temp_list= list()
            for i in range(len(a)):
                temp_list.append((a[i],b[i]))
            return temp_list
    sentences = list()
    for idx, row in df.iterrows():
        zipped_sentence = tuple(zip_(eval(row.token),eval(row.ner)))
        sentences.append(zipped_sentence)
    return sentences

In [57]:
sentences_train = getsentence(train_df)
sentences_test = getsentence(test_df)
print(sentences_train[:1])
print(sentences_test[:1])


[(('@ridwankamil', 'O'), ('@KickAndyShow', 'O'), ('@Metro_TV', 'O'), ('Upaya', 'O'), ('untuk', 'O'), ('mnaikan', 'O'), ('nilai', 'O'), ('jual', 'O'), ('yg', 'O'), ('bgus,dngn', 'O'), ('cara', 'O'), ('d', 'O'), ('desain', 'O'), ('pak', 'O'), ('gubernur', 'B-PER'), ('akan', 'O'), ('menjadikn', 'O'), ('produk', 'O'), ('umkm', 'O'), ('d', 'O'), ('mintai', 'O'), ('krena', 'O'), ('yg', 'O'), ('desain', 'O'), ('org', 'O'), ('nomer', 'O'), ('satu', 'O'), ('di', 'O'), ('jabar', 'B-LOC'))]
[(('@GyuuPotter', 'O'), ('iyhh', 'O'), (',', 'O'), ('soalnya', 'O'), ('mwu', 'O'), ('kerjain', 'O'), ('tugas', 'O'), ('jg��', 'O'))]


In [63]:
# words = list(train_df["token"]))
# n_words = len(words)
# print(n_words)

5809


In [67]:
# Feature set
def word2features(sent, i):
    word = sent[i][0]
    # we=get_features(word) 
    # postag = sent[i][1]

    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower() if type(word) == str else word,
        'word[-3:]': word[-3:] if type(word) == str else word,
        'word[-2:]': word[-2:] if type(word) == str else word,
        'word[-1:]': word[-2:] if type(word) == str else word,
        'word[:3]': word[:3] if type(word) == str else word,
        'word[:2]': word[:2] if type(word) == str else word,
        'word[:1]': word[:2] if type(word) == str else word,
        'word.isupper()': word.isupper() if type(word) == str else word,
        'word.istitle()': word.istitle() if type(word) == str else word,
        'word.isdigit()': word.isdigit() if type(word) == str else word,
        # 'postag': postag,
        # 'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        # postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower() if type(word1) == str else word1,
            '-1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '-1:word.isupper()': word1.isupper() if type(word1) == str else word1,
            # '-1:postag': postag1,
            # '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        # postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower() if type(word1) == str else word1,
            '+1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '+1:word.isupper()': word1.isupper() if type(word1) == str else word1,
            # '+1:postag': postag1,
            # '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    #src: https://stackoverflow.com/questions/58736548/how-to-use-word-embedding-as-features-for-crf-sklearn-crfsuite-model-training
    # we=model.get_word_vector(features['word.lower()'])
    # for iv,value in enumerate(we):
    #     features['v{}'.format(iv)]=value

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [ ]:
# Feature set
def word2featuresWE(sent, i):
    word = sent[i][0]
    we=get_features(word) 
    postag = sent[i][1]

    
    features = {
        'bias': 1.0,
        'word.lower()': word.lower() if type(word) == str else word,
        'word[-3:]': word[-3:] if type(word) == str else word,
        'word[-2:]': word[-2:] if type(word) == str else word,
        'word[-1:]': word[-2:] if type(word) == str else word,
        'word[:3]': word[:3] if type(word) == str else word,
        'word[:2]': word[:2] if type(word) == str else word,
        'word[:1]': word[:2] if type(word) == str else word,
        'word.isupper()': word.isupper() if type(word) == str else word,
        'word.istitle()': word.istitle() if type(word) == str else word,
        'word.isdigit()': word.isdigit() if type(word) == str else word,
        # 'postag': postag,
        # 'postag[:2]': postag[:2],
    }
    if i > 0:
        word1 = sent[i-1][0]
        # postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower() if type(word1) == str else word1,
            '-1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '-1:word.isupper()': word1.isupper() if type(word1) == str else word1,
            # '-1:postag': postag1,
            # '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True

    if i < len(sent)-1:
        word1 = sent[i+1][0]
        # postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower() if type(word1) == str else word1,
            '+1:word.istitle()': word1.istitle() if type(word1) == str else word1,
            '+1:word.isupper()': word1.isupper() if type(word1) == str else word1,
            # '+1:postag': postag1,
            # '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    #src: https://stackoverflow.com/questions/58736548/how-to-use-word-embedding-as-features-for-crf-sklearn-crfsuite-model-training
    # we=model.get_word_vector(features['word.lower()'])
    # for iv,value in enumerate(we):
    #     features['v{}'.format(iv)]=value

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2labels(sent):
    return [label for token, label in sent]

In [68]:
%%time
#Creating the train and test set
X_train = [sent2features(s) for s in sentences_train]
X_test = [sent2features(s) for s in sentences_test]
y_train = [sent2labels(s) for s in sentences_train]
y_test = [sent2labels(s) for s in sentences_test]

Wall time: 1.3 s


In [70]:
# group B and I results
# #First we select all the tags that are relevant for us i.e. remove the 'O' tag from the list. 

# labels = list(crf2.classes_)
# labels = list(train_df['labels'].unique())
# labels = list(filter(lambda a: a != 'O', labels))
# sorted_labels = sorted(
#     labels,
#     key=lambda name: (name[1:], name[0])
# )
sorted_labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA']
print(sorted_labels)

['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA']


In [71]:
#Creating the CRF model
crf = CRF(algorithm='lbfgs',
          c1= 0.05912981852829489, 
          c2= 0.09113455878833218,
          max_iterations=100,
          all_possible_transitions=False)

In [72]:
%%time
pred = cross_val_predict(estimator=crf, X=X_train, y=y_train, cv=5)
print(flat_classification_report(y_pred=pred, y_true=y_train, labels=sorted_labels, digits=3))   

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)
<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


              precision    recall  f1-score   support

        B-EV      0.602     0.250     0.353       661
        I-EV      0.569     0.317     0.407       779
       B-LOC      0.765     0.626     0.688      1788
       I-LOC      0.637     0.401     0.492       761
       B-ORG      0.641     0.384     0.480      1696
       I-ORG      0.558     0.374     0.448       931
       B-PER      0.867     0.641     0.737      4416
       I-PER      0.842     0.723     0.778      2455
      B-PROD      0.876     0.733     0.798      2878
      I-PROD      0.749     0.448     0.560       934
        B-WA      0.588     0.074     0.131       136
        I-WA      0.286     0.036     0.063       224

   micro avg      0.788     0.566     0.659     17659
   macro avg      0.665     0.417     0.495     17659
weighted avg      0.768     0.566     0.646     17659

Wall time: 2min 32s


In [73]:
#Tuning the parameters manually, setting c1 = 10
crf2 = CRF(algorithm='lbfgs',
          c1=1,
          c2=0.1,
          max_iterations=100,
          all_possible_transitions=False)

In [74]:
%%time
pred = cross_val_predict(estimator=crf2, X=X_train, y=y_train, cv=5)
print(flat_classification_report(y_pred=pred, y_true=y_train, labels=sorted_labels, digits=3))  

              precision    recall  f1-score   support

        B-EV      0.686     0.201     0.311       661
        I-EV      0.548     0.262     0.354       779
       B-LOC      0.762     0.595     0.668      1788
       I-LOC      0.610     0.399     0.483       761
       B-ORG      0.651     0.340     0.446      1696
       I-ORG      0.540     0.374     0.442       931
       B-PER      0.877     0.616     0.724      4416
       I-PER      0.832     0.715     0.769      2455
      B-PROD      0.882     0.715     0.790      2878
      I-PROD      0.719     0.433     0.540       934
        B-WA      0.000     0.000     0.000       136
        I-WA      0.000     0.000     0.000       224

   micro avg      0.789     0.542     0.642     17659
   macro avg      0.592     0.387     0.461     17659
weighted avg      0.761     0.542     0.627     17659

Wall time: 2min 32s


In [76]:
%%time
#Now we will create the Randomized CV search model wherein we will use a modified F1 scorer model considering only the relevant labels
# define fixed parameters and parameters to search
crf3 = CRF(
    algorithm='lbfgs',
    max_iterations=100,
    all_possible_transitions=True
)   
params_space = {
    'c1': scipy.stats.expon(scale=0.5),
    'c2': scipy.stats.expon(scale=0.05),
}

# use the same metric for evaluation
f1_scorer = make_scorer(metrics.flat_f1_score,
                        average='weighted', labels=sorted_labels)

# search
rs = RandomizedSearchCV(crf, params_space,
                        cv=3,
                        verbose=1,
                        n_jobs=8, #-1
                        n_iter=30,#50,
                        scoring=f1_scorer)
rs.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:  9.2min
[Parallel(n_jobs=8)]: Done  90 out of  90 | elapsed: 23.3min finished


Wall time: 23min 51s


RandomizedSearchCV(cv=3,
                   estimator=CRF(algorithm='lbfgs',
                                 all_possible_transitions=False,
                                 c1=0.05912981852829489, c2=0.09113455878833218,
                                 keep_tempfiles=None, max_iterations=100),
                   n_iter=30, n_jobs=8,
                   param_distributions={'c1': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000016F253E7308>,
                                        'c2': <scipy.stats._distn_infrastructure.rv_frozen object at 0x0000016F2F0B4288>},
                   scoring=make_scorer(flat_f1_score, average=weighted, labels=['B-EV', 'I-EV', 'B-LOC', 'I-LOC', 'B-ORG', 'I-ORG', 'B-PER', 'I-PER', 'B-PROD', 'I-PROD', 'B-WA', 'I-WA']),
                   verbose=1)

In [77]:
#Lets check the best estimated parameters and CV score
print('Best parameters:', rs.best_params_)
print('Best CV score:', rs.best_score_)
print('Model size: {:0.2f}M'.format(rs.best_estimator_.size_ / 1000000))

Best parameters: {'c1': 0.15916848200144398, 'c2': 0.179379974255535}
Best CV score: 0.6258468419300472
Model size: 2.66M


In [78]:
#Now we create the model again using the best estimators
crf3 = rs.best_estimator_
y_pred = crf3.predict(X_test)
print(metrics.flat_classification_report(y_test, y_pred, labels=sorted_labels, digits=3))

              precision    recall  f1-score   support

        B-EV      0.621     0.277     0.383       148
        I-EV      0.625     0.352     0.450       199
       B-LOC      0.800     0.696     0.744       437
       I-LOC      0.733     0.561     0.636       196
       B-ORG      0.663     0.434     0.525       389
       I-ORG      0.562     0.513     0.537       193
       B-PER      0.930     0.687     0.791      1148
       I-PER      0.914     0.806     0.856       603
      B-PROD      0.889     0.797     0.841       676
      I-PROD      0.746     0.543     0.629       243
        B-WA      0.667     0.071     0.129        28
        I-WA      0.545     0.128     0.207        47

   micro avg      0.828     0.638     0.721      4307
   macro avg      0.725     0.489     0.561      4307
weighted avg      0.818     0.638     0.711      4307



In [79]:
%%time
pred = cross_val_predict(estimator=crf3, X=X_test, y=y_test, cv=5)
print(flat_classification_report(y_pred=pred, y_true=y_test, labels=sorted_labels, digits=3))

d:\Drive\KULEEAH\Semester 8\TA Farihin\env\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

        B-EV      0.491     0.189     0.273       148
        I-EV      0.638     0.221     0.328       199
       B-LOC      0.760     0.551     0.639       437
       I-LOC      0.511     0.357     0.420       196
       B-ORG      0.604     0.239     0.343       389
       I-ORG      0.441     0.212     0.287       193
       B-PER      0.908     0.565     0.697      1148
       I-PER      0.866     0.698     0.773       603
      B-PROD      0.886     0.621     0.730       676
      I-PROD      0.644     0.276     0.386       243
        B-WA      0.000     0.000     0.000        28
        I-WA      0.000     0.000     0.000        47

   micro avg      0.796     0.482     0.600      4307
   macro avg      0.562     0.328     0.406      4307
weighted avg      0.760     0.482     0.583      4307

Wall time: 35 s


In [82]:
pred

array([list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
       list(['O', 'O', 'O', 'O', 'O', 'B-PROD', 'O', 'O', 'O', 'O']),
       list(['O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
       ..., list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']),
       list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O', 'O']),
       list(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'O', 'O', 'O', 'O'])],
      dtype=object)

In [83]:
y_test

[['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'O', 'O', 'O', 'O', 'B-PROD', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'O',
  'O',
  'B-PER',
  'I-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['B-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'B-PER',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  

In [80]:
crf3.fit(X_train,y_train)

CRF(algorithm='lbfgs', all_possible_transitions=False, c1=0.15916848200144398,
    c2=0.179379974255535, keep_tempfiles=None, max_iterations=100)

In [81]:
eli5.show_weights(crf3, top=30)

From \ To,O,B-EV,I-EV,B-LOC,I-LOC,B-ORG,I-ORG,B-PER,I-PER,B-PROD,I-PROD,B-WA,I-WA
O,2.535,0.972,-1.764,1.289,-2.035,2.19,-2.138,2.181,-3.041,1.907,-1.603,1.228,-1.509
B-EV,-0.64,2.046,6.489,-0.535,0.0,0.588,0.0,-0.15,0.0,1.08,0.0,0.0,0.0
I-EV,-0.18,0.877,6.395,-0.776,0.0,-0.558,0.0,-0.233,0.0,0.0,0.0,0.0,0.0
B-LOC,0.681,0.0,0.0,1.84,6.072,0.367,0.0,-0.916,0.0,0.303,0.0,0.232,0.0
I-LOC,-0.289,0.0,0.0,0.519,5.777,0.0,0.0,-0.649,0.0,0.0,0.0,0.0,0.0
B-ORG,0.49,0.915,0.0,-0.302,0.0,0.934,5.615,-0.369,0.0,-0.07,0.0,1.086,0.0
I-ORG,0.065,0.0,0.0,-1.581,0.0,0.162,5.501,-0.97,0.0,0.075,0.0,0.0,0.0
B-PER,1.197,0.0,0.0,0.918,0.0,0.785,0.0,2.104,5.789,0.74,0.0,0.0,0.0
I-PER,0.287,0.0,0.0,-0.126,0.0,0.457,0.0,-0.678,4.043,0.0,0.0,0.0,0.0
B-PROD,0.494,-0.021,0.0,-0.559,0.0,0.556,0.0,-0.324,0.0,2.178,5.302,0.0,0.0
